# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

In [40]:
import dask.array as da
import h5py
import numpy as np
import dask

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [41]:
%%time
arr_np = np.random.normal(0,1, size = (1000,300_000))

CPU times: user 6.88 s, sys: 476 ms, total: 7.36 s
Wall time: 7.4 s


In [42]:
%%time
arr_da = da.random.normal(0,1, size = (1000,300_000),chunks = (1000,30_000))

CPU times: user 6.09 ms, sys: 4.71 ms, total: 10.8 ms
Wall time: 12.8 ms


In [43]:
arr_da

dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [44]:
%%time
np.power(arr_np, 2).sum()

CPU times: user 6.78 s, sys: 1.74 s, total: 8.51 s
Wall time: 9.66 s


300008142.0113529

In [45]:
%%time
da.power(arr_da, 2).sum().compute()

CPU times: user 15.9 s, sys: 3.1 s, total: 19 s
Wall time: 4.36 s


300036883.05013835

In [10]:
%%time
da.power(arr_da.compute(), 2).sum()

CPU times: user 15.3 s, sys: 2.99 s, total: 18.3 s
Wall time: 12.4 s


300018484.2142836

3. Визуализируйте граф вычислений для задачи 2.

In [15]:
pip install graphviz

     |████████████████████████████████| 46 kB 678 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [39]:
from dask.dot import dot_graph
da.power(arr_da, 2).sum()

NameError: name 'arr_da' is not defined

## Лабораторная работа 11

In [1]:
import dask.array as da
import h5py
import numpy as np

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [128]:
hdf = h5py.File('data/minutes_n_ingredients_full.hdf5', 'r')
data = da.from_array(hdf['recipe'],chunks = (100_000, 3))
data

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [30]:
data[:,1:].mean(axis = 0).compute()

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [31]:
%%time
data = da.from_array(hdf['recipe'],chunks = (10000, 3))
data[:,1:].mean(axis = 0).compute()

CPU times: user 187 ms, sys: 22.8 ms, total: 210 ms
Wall time: 176 ms


array([1004.20805176,    5.4198008 ])

In [32]:
%%time
data = da.from_array(hdf['recipe'],chunks = (100000, 3))
data[:,1:].mean(axis = 0).compute()

CPU times: user 105 ms, sys: 12.6 ms, total: 117 ms
Wall time: 74.1 ms


array([1004.20805176,    5.4198008 ])

In [33]:
%%time
data = da.from_array(hdf['recipe'],chunks = (1000000, 3))
data[:,1:].mean(axis = 0).compute()

CPU times: user 89.9 ms, sys: 25.3 ms, total: 115 ms
Wall time: 93.1 ms


array([1004.20805176,    5.4198008 ])

In [44]:
%%time
data = da.from_array(hdf['recipe'],chunks = (500000, 3))
data[:,1:].mean(axis = 0).compute()

CPU times: user 95.5 ms, sys: 12.2 ms, total: 108 ms
Wall time: 65.7 ms


array([1004.20805176,    5.4198008 ])

In [43]:
%%time
data = da.from_array(hdf['recipe'],chunks = (100000, 81))
data[:,1:].mean(axis = 0).compute()

CPU times: user 104 ms, sys: 14.3 ms, total: 118 ms
Wall time: 74.3 ms


array([1004.20805176,    5.4198008 ])

In [ ]:
#есть некая золотая середина в виде (500000,3) которая дает самую большую скорость - меньше и больше скорость падает

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [51]:
pip install --upgrade dask

     |████████████████████████████████| 1.0 MB 3.0 MB/s eta 0:00:01
  Attempting uninstall: dask
    Found existing installation: dask 2021.4.0
    Uninstalling dask-2021.4.0:
      Successfully uninstalled dask-2021.4.0
Note: you may need to restart the kernel to use updated packages.


In [12]:
data = da.from_array(hdf['recipe'],chunks = (100000, 3))
data[data[:,2] < (da.median(data[:,2],axis = 0))].compute()

array([[1400250,      24,       1],
       [1091642,      49,       1],
       [1121713,      11,       4],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [1910650,      60,       2]])

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [21]:
da.unique(data[:,2]).compute().shape[0]

41

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [23]:
da.max(data[:,1]).compute()

2147483647

In [131]:
data[data[:,1]<da.percentile(data[:,1],75)].compute()

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]])

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [52]:
exmpl = da.from_array([1000,10])
data[da.argmin(da.sum(da.absolute(data[:,1:] - exmpl),axis = 1))].compute()

array([13388,  1000,     9])

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [138]:
%%time
hdf = h5py.File('data/minutes_n_ingredients_full.hdf5', 'r')
block_size = 10000
stop_flag = int(hdf['recipe'].shape[0]/block_size) + 1
length = hdf['recipe'].shape[0]
counts = np.zeros(stop_flag)
for block_counter in range(stop_flag):
    cur = hdf['recipe'][(block_counter*block_size):((block_counter+1)*block_size),1]
    counts[block_counter] = cur.mean()*cur.shape[0]
hdf.close()
sum(counts)/length

CPU times: user 93.3 ms, sys: 11.3 ms, total: 105 ms
Wall time: 105 ms


1004.2080517575215

In [113]:
%%time
hdf = h5py.File('data/minutes_n_ingredients_full.hdf5', 'r')
hdf['recipe'][:,1].mean()

CPU times: user 29 ms, sys: 9.15 ms, total: 38.2 ms
Wall time: 39 ms


1004.2080517575215